In [1]:
#DAILY PERCENTAGE RETURNS DATA... need this to see how the price reacts to a given news release
# Remember, we are just doing the daily percentage gains, not going intraday yet, to keep things simple.

#Let's set up our Alpaca API...

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
%matplotlib inline



In [2]:
# Load .env environment variables
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Verify that Alpaca key and secret were correctly loaded without compromising security
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [4]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
# Create our timeframe: Jan 1, 2022 - Dec 31, 2022
# Format date as ISO format
start_date = pd.Timestamp("2022-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-12-31", tz="America/New_York").isoformat()

In [6]:
# We need to get the tickers for all stocks in the S&P 500
# Going to get the symbols of the stocks contained in the S&P 500 from the dataframe created in "DataCleaning.ipynb"

# Load in libraries needed to read the csv
import pandas as pd
from pathlib import Path

# Defining Paths
snp500_path = Path("../Group3Project1/Resources/constituents.csv")

# Reading CSV
snp500_data = pd.read_csv(snp500_path, index_col="Symbol")

# Sort data in alphabetical order. We will keep this consistient among the datasets we create.
snp500_data.sort_index(inplace=True)

# Preview Data
display(snp500_data.head())
display(snp500_data.tail())

,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Symbol,,,,,,,
A,Agilent Technologies,Health Care,Health Care Equipment,"Santa Clara, California",2000-06-05,1090872,1999
AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas",2015-03-23,6201,1934
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977
ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines","San Francisco, California",2023-09-18,1559720,2008


,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Symbol,,,,,,,
YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873
ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [7]:
# Our goal here is to capture the "Symbol" column of the snp500 dataframe.

# Reset the index so that we have Symbol as a normal column. Save the result of that operation with "inplace=True".
snp500_data.reset_index(inplace=True)

# Preview data.
snp500_data.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Health Care Equipment,"Santa Clara, California",2000-06-05,1090872,1999
1,AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas",2015-03-23,6201,1934
2,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines","San Francisco, California",2023-09-18,1559720,2008


In [8]:
# Now we can capture the symbol column
snp500_symbols = snp500_data["Symbol"]

# Preview data.
display(snp500_symbols.head())
display(snp500_symbols.tail())

0       A
1     AAL
2    AAPL
3    ABBV
4    ABNB
Name: Symbol, dtype: object

498     YUM
499     ZBH
500    ZBRA
501    ZION
502     ZTS
Name: Symbol, dtype: object

NOTICE: Why do we have 503 rows? Isn't it the S&P *500*? 

Although the it is called the "S&P 500", it constitutes of 503 stocks.


In [9]:
# Now that we have an array of all symbols in the S&P 500, we want to use that list to get data for those tickers using Alpaca API


# Given the Alpaca API's protocal, we should set 'tickers' to all stocks in the S&P 500. 

# Need to include '[:]' or else the snp500_symbols will be set equal to "tickers"...
# which would be troublesome because we need to keep the snp500_symbols index the same for later use.
tickers = snp500_symbols[:]

# We want to increment the index by 1, because the Alpaca API starts taking in data starting at Index = 1... I THINK.. CHECK THIS!!
tickers.index = tickers.index+1

display(tickers.head())

print('Origional')
display(snp500_symbols.head())

1       A
2     AAL
3    AAPL
4    ABBV
5    ABNB
Name: Symbol, dtype: object

Origional


0       A
1     AAL
2    AAPL
3    ABBV
4    ABNB
Name: Symbol, dtype: object

In [10]:
# Notice that the above indicies are begin at 1 and 0, respectively, which is good.

In [11]:
# We are fousing on daily returns.
# Set timeframe to one day ("1Day") for the Alpaca API.
timeframe = "1Day"

# Get daily closing prices for year 2022 of all stocks in S&P 500.
snp500_price_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Display sample data
snp500_price_df.head()


,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-01-03 05:00:00+00:00,18.75,19.1800,18.2000,162182,18.230,44860486,18.777172,AAL
2022-01-04 05:00:00+00:00,19.02,19.4600,18.8504,119441,19.215,30433127,19.089447,AAL
2022-01-05 05:00:00+00:00,18.68,19.3850,18.5100,133200,19.130,35253129,18.917920,AAL
2022-01-06 05:00:00+00:00,18.57,19.0599,18.3500,91484,18.890,19726860,18.669406,AAL
2022-01-07 05:00:00+00:00,19.28,19.5900,18.5800,158007,18.750,43826540,19.210886,AAL


In [12]:
# Notice that above the first symbol listed is "AAL", however, we need "A" to be first, as that is the order of our initial symbol list.

# Let's play around with the index so that we can properely sort our new DF.

# Reset the index
snp500_price_df = snp500_price_df.reset_index()

display(snp500_price_df.head())
display(snp500_price_df.tail())


,timestamp,close,high,low,trade_count,open,volume,vwap,symbol
0,2022-01-03 05:00:00+00:00,18.75,19.1800,18.2000,162182,18.230,44860486,18.777172,AAL
1,2022-01-04 05:00:00+00:00,19.02,19.4600,18.8504,119441,19.215,30433127,19.089447,AAL
2,2022-01-05 05:00:00+00:00,18.68,19.3850,18.5100,133200,19.130,35253129,18.917920,AAL
3,2022-01-06 05:00:00+00:00,18.57,19.0599,18.3500,91484,18.890,19726860,18.669406,AAL
4,2022-01-07 05:00:00+00:00,19.28,19.5900,18.5800,158007,18.750,43826540,19.210886,AAL


,timestamp,close,high,low,trade_count,open,volume,vwap,symbol
125422,2022-12-23 05:00:00+00:00,145.76,145.8920,143.5400,20025,144.51,1017604,145.236784,ZTS
125423,2022-12-27 05:00:00+00:00,145.30,146.1474,143.5700,19294,145.91,957871,145.322438,ZTS
125424,2022-12-28 05:00:00+00:00,143.83,146.6400,143.7700,19669,145.18,1443916,144.409503,ZTS
125425,2022-12-29 05:00:00+00:00,148.15,148.5100,145.1378,21072,145.20,1298851,147.959023,ZTS
125426,2022-12-30 05:00:00+00:00,146.55,147.7900,144.7400,22281,147.20,1249518,146.129091,ZTS


In [13]:
# Set the index to 'symbol' so we can sort alphabetically by 'symbol'
snp500_price_df = snp500_price_df.set_index('symbol')
display(snp500_price_df.head())
display(snp500_price_df.tail())

,timestamp,close,high,low,trade_count,open,volume,vwap
symbol,,,,,,,,
AAL,2022-01-03 05:00:00+00:00,18.75,19.1800,18.2000,162182,18.230,44860486,18.777172
AAL,2022-01-04 05:00:00+00:00,19.02,19.4600,18.8504,119441,19.215,30433127,19.089447
AAL,2022-01-05 05:00:00+00:00,18.68,19.3850,18.5100,133200,19.130,35253129,18.917920
AAL,2022-01-06 05:00:00+00:00,18.57,19.0599,18.3500,91484,18.890,19726860,18.669406
AAL,2022-01-07 05:00:00+00:00,19.28,19.5900,18.5800,158007,18.750,43826540,19.210886


,timestamp,close,high,low,trade_count,open,volume,vwap
symbol,,,,,,,,
ZTS,2022-12-23 05:00:00+00:00,145.76,145.8920,143.5400,20025,144.51,1017604,145.236784
ZTS,2022-12-27 05:00:00+00:00,145.30,146.1474,143.5700,19294,145.91,957871,145.322438
ZTS,2022-12-28 05:00:00+00:00,143.83,146.6400,143.7700,19669,145.18,1443916,144.409503
ZTS,2022-12-29 05:00:00+00:00,148.15,148.5100,145.1378,21072,145.20,1298851,147.959023
ZTS,2022-12-30 05:00:00+00:00,146.55,147.7900,144.7400,22281,147.20,1249518,146.129091


In [14]:
# Now sort index alphabetically, ascending is the default. 
snp500_price_df = snp500_price_df.sort_index()
display(snp500_price_df.head())
display(snp500_price_df.tail())

,timestamp,close,high,low,trade_count,open,volume,vwap
symbol,,,,,,,,
A,2022-01-03 05:00:00+00:00,156.48,159.440,153.930,24320,159.00,1855491,155.916852
A,2022-08-23 04:00:00+00:00,132.64,134.105,132.044,22511,133.11,1312701,132.799144
A,2022-08-24 04:00:00+00:00,133.67,134.540,132.650,19162,133.02,1171108,133.682537
A,2022-08-25 04:00:00+00:00,136.01,136.080,133.990,18936,135.30,1082407,135.147622
A,2022-08-26 04:00:00+00:00,129.87,136.050,129.780,23682,136.00,1439050,131.184323


,timestamp,close,high,low,trade_count,open,volume,vwap
symbol,,,,,,,,
ZTS,2022-05-10 04:00:00+00:00,162.65,164.75,159.08,53452,163.99,3141027,161.982615
ZTS,2022-05-11 04:00:00+00:00,159.90,164.11,159.26,46055,161.87,4239464,160.763873
ZTS,2022-05-12 04:00:00+00:00,163.08,163.68,158.27,41201,158.34,2672868,162.002346
ZTS,2022-04-25 04:00:00+00:00,180.30,181.11,176.44,39876,180.69,2384476,178.997726
ZTS,2022-12-30 05:00:00+00:00,146.55,147.79,144.74,22281,147.20,1249518,146.129091


In [15]:
# We see above that we are now sorted correctly.


# Now we want to reset the index, so we recreate 'timestamp' as the index.
snp500_price_df = snp500_price_df.reset_index()


In [16]:
# Ensure the reset is proper.
snp500_price_df

,symbol,timestamp,close,high,low,trade_count,open,volume,vwap
0,A,2022-01-03 05:00:00+00:00,156.48,159.440,153.930,24320,159.00,1855491,155.916852
1,A,2022-08-23 04:00:00+00:00,132.64,134.105,132.044,22511,133.11,1312701,132.799144
2,A,2022-08-24 04:00:00+00:00,133.67,134.540,132.650,19162,133.02,1171108,133.682537
3,A,2022-08-25 04:00:00+00:00,136.01,136.080,133.990,18936,135.30,1082407,135.147622
4,A,2022-08-26 04:00:00+00:00,129.87,136.050,129.780,23682,136.00,1439050,131.184323
...,...,...,...,...,...,...,...,...,...
125422,ZTS,2022-05-10 04:00:00+00:00,162.65,164.750,159.080,53452,163.99,3141027,161.982615
125423,ZTS,2022-05-11 04:00:00+00:00,159.90,164.110,159.260,46055,161.87,4239464,160.763873
125424,ZTS,2022-05-12 04:00:00+00:00,163.08,163.680,158.270,41201,158.34,2672868,162.002346
125425,ZTS,2022-04-25 04:00:00+00:00,180.30,181.110,176.440,39876,180.69,2384476,178.997726


In [17]:
# Return 'timestamp' as the index.
snp500_price_df = snp500_price_df.set_index('timestamp')

In [18]:
# Now the df is properly sorted by 'symbol' and the 'timestamp' is the index again.
display(snp500_price_df.head())
display(snp500_price_df.tail())

,symbol,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,,
2022-01-03 05:00:00+00:00,A,156.48,159.440,153.930,24320,159.00,1855491,155.916852
2022-08-23 04:00:00+00:00,A,132.64,134.105,132.044,22511,133.11,1312701,132.799144
2022-08-24 04:00:00+00:00,A,133.67,134.540,132.650,19162,133.02,1171108,133.682537
2022-08-25 04:00:00+00:00,A,136.01,136.080,133.990,18936,135.30,1082407,135.147622
2022-08-26 04:00:00+00:00,A,129.87,136.050,129.780,23682,136.00,1439050,131.184323


,symbol,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,,
2022-05-10 04:00:00+00:00,ZTS,162.65,164.75,159.08,53452,163.99,3141027,161.982615
2022-05-11 04:00:00+00:00,ZTS,159.90,164.11,159.26,46055,161.87,4239464,160.763873
2022-05-12 04:00:00+00:00,ZTS,163.08,163.68,158.27,41201,158.34,2672868,162.002346
2022-04-25 04:00:00+00:00,ZTS,180.30,181.11,176.44,39876,180.69,2384476,178.997726
2022-12-30 05:00:00+00:00,ZTS,146.55,147.79,144.74,22281,147.20,1249518,146.129091


Now we need to reorganize the dataframe so that each symbol is its own dataframe.


In [19]:
# We will essentially be doing the following for each symbol...
A = snp500_price_df[snp500_price_df['symbol'] == 'A'].drop('symbol', axis=1)
ZTS = snp500_price_df[snp500_price_df['symbol'] == 'ZTS'].drop('symbol', axis=1)

# Then we concatenate those symbols, using the ticker as the key.
test_df_A_ZTS = pd.concat([A, ZTS], axis=1, keys=['A', 'ZTS'])

display(test_df_A_ZTS.head())

A                                      \
                            close    high     low trade_count    open   
timestamp                                                               
2022-01-03 05:00:00+00:00  156.48  159.44  153.93       24320  159.00   
2022-01-04 05:00:00+00:00  151.19  155.63  149.70       34709  155.49   
2022-01-05 05:00:00+00:00  148.60  153.10  148.53       27423  150.83   
2022-01-06 05:00:00+00:00  149.12  149.96  145.58       34443  148.85   
2022-01-07 05:00:00+00:00  145.15  149.73  145.09       28610  149.12   

                                                   ZTS                   \
                            volume        vwap   close    high      low   
timestamp                                                                 
2022-01-03 05:00:00+00:00  1855491  155.916852  234.03  243.24  233.320   
2022-01-04 05:00:00+00:00  2510184  151.522046  225.12  234.02  222.140   
2022-01-05 05:00:00+00:00  2966289  149.705613  216.56  227.00  215.600   
2022-01-06 05:00:00+00:00  2502203  148.493000  217.42  220.56  213.625   
2022-01-07 05:00:00+00:00  2415233  146.271401  211.09  217.42  210.780   

                                                                    
                          trade_count    open   volume        vwap  
timestamp                                                           
2022-01-03 05:00:00+00:00       40631  242.29  3110633  234.902990  
2022-01-04 05:00:00+00:00       72176  234.02  5082693  225.646202  
2022-01-05 05:00:00+00:00       74079  227.00  5352738  218.720396  
2022-01-06 05:00:00+00:00       53574  215.48  3448961  217.727004  
2022-01-07 05:00:00+00:00       35901  217.13  2584840  212.467092

Let's go ahead and iterate this.

In [20]:
# First, create a list of symbols so that we can set the keys easily when creating the final df.
# Use the origional array of symbols we have above 'snp500_symbols'

# Create empty list.
snp500_symbols_list=[]

# Given length of our array 'snp500_symbols', add each element of that array to our new list, 'snp500_symbols_list'
for x in range(len(snp500_symbols)):
    snp500_symbols_list.append(snp500_symbols[x])

# Preview list and its length
display(snp500_symbols_list[:5])
display(snp500_symbols_list[-5:])
len(snp500_symbols_list)

['A', 'AAL', 'AAPL', 'ABBV', 'ABNB']

['YUM', 'ZBH', 'ZBRA', 'ZION', 'ZTS']

503

In [21]:
# Now that we have a list of symbols, lets start creating the new dataframe with each symbol having its own set of data.

snp500_2022_returns_df = pd.DataFrame()

# We will create a list of these dataframes for easier concatenation. 
df_list = []


# Make list of the dataframes. Each df is the data associated with one symbol in our snp500_price_df

# For all elements('symbols') in 'snp500_symbols_list'...
for x in snp500_symbols_list:
    # Create a new element ('ticker_data_1'), which is a dataframe the data associated with one symbol.
    ticker_data_1 = snp500_price_df[snp500_price_df['symbol'] == x].drop('symbol', axis=1)
    # Add that element to our list of dataframes
    df_list.append(ticker_data_1)    

# Concat the list of dataframes. Set the keys of each df to the list of symbols that we created.
snp500_2022_returns_df = pd.concat(df_list, axis=1, keys=snp500_symbols_list)

In [31]:
# Preview dataframe
# I am including both ends of the data for the next step; double checking to see if first/last values align.
display(snp500_2022_returns_df.head())
display(snp500_2022_returns_df.tail())

A                                      \
                            close    high     low trade_count    open   
timestamp                                                               
2022-01-03 05:00:00+00:00  156.48  159.44  153.93       24320  159.00   
2022-01-04 05:00:00+00:00  151.19  155.63  149.70       34709  155.49   
2022-01-05 05:00:00+00:00  148.60  153.10  148.53       27423  150.83   
2022-01-06 05:00:00+00:00  149.12  149.96  145.58       34443  148.85   
2022-01-07 05:00:00+00:00  145.15  149.73  145.09       28610  149.12   

                                                  AAL                    ...  \
                            volume        vwap  close     high      low  ...   
timestamp                                                                ...   
2022-01-03 05:00:00+00:00  1855491  155.916852  18.75  19.1800  18.2000  ...   
2022-01-04 05:00:00+00:00  2510184  151.522046  19.02  19.4600  18.8504  ...   
2022-01-05 05:00:00+00:00  2966289  149.705613  18.68  19.3850  18.5100  ...   
2022-01-06 05:00:00+00:00  2502203  148.493000  18.57  19.0599  18.3500  ...   
2022-01-07 05:00:00+00:00  2415233  146.271401  19.28  19.5900  18.5800  ...   

                             ZION                         ZTS          \
                             open   volume       vwap   close    high   
timestamp                                                               
2022-01-03 05:00:00+00:00  63.880  1414361  64.236825  234.03  243.24   
2022-01-04 05:00:00+00:00  66.110  1557604  66.314555  225.12  234.02   
2022-01-05 05:00:00+00:00  66.465  1121305  66.574460  216.56  227.00   
2022-01-06 05:00:00+00:00  67.100  1660365  68.300571  217.42  220.56   
2022-01-07 05:00:00+00:00  69.180  1456668  69.918369  211.09  217.42   

                                                                             
                               low trade_count    open   volume        vwap  
timestamp                                                                    
2022-01-03 05:00:00+00:00  233.320       40631  242.29  3110633  234.902990  
2022-01-04 05:00:00+00:00  222.140       72176  234.02  5082693  225.646202  
2022-01-05 05:00:00+00:00  215.600       74079  227.00  5352738  218.720396  
2022-01-06 05:00:00+00:00  213.625       53574  215.48  3448961  217.727004  
2022-01-07 05:00:00+00:00  210.780       35901  217.13  2584840  212.467092  

[5 rows x 3521 columns]

A                                              \
                            close    high     low trade_count    open  volume   
timestamp                                                                       
2022-12-23 05:00:00+00:00  149.23  150.25  148.22       17530  149.01  779426   
2022-12-27 05:00:00+00:00  149.55  149.81  147.30       17148  149.23  879543   
2022-12-28 05:00:00+00:00  148.09  151.37  147.64       19078  149.78  784312   
2022-12-29 05:00:00+00:00  151.09  151.90  149.02       17820  149.61  854392   
2022-12-30 05:00:00+00:00  149.65  149.85  147.83       15040  149.76  699759   

                                         AAL                   ...   ZION  \
                                 vwap  close    high      low  ...   open   
timestamp                                                      ...          
2022-12-23 05:00:00+00:00  149.203604  12.71  12.710  12.3298  ...  48.28   
2022-12-27 05:00:00+00:00  149.313607  12.53  12.790  12.3950  ...  48.65   
2022-12-28 05:00:00+00:00  148.753919  12.32  12.680  12.2500  ...  48.87   
2022-12-29 05:00:00+00:00  151.121691  12.70  12.710  12.2950  ...  48.06   
2022-12-30 05:00:00+00:00  149.211086  12.72  12.825  12.5300  ...  48.61   

                                                  ZTS                      \
                            volume       vwap   close      high       low   
timestamp                                                                   
2022-12-23 05:00:00+00:00  1354480  48.435983  145.76  145.8920  143.5400   
2022-12-27 05:00:00+00:00   795675  48.808658  145.30  146.1474  143.5700   
2022-12-28 05:00:00+00:00   776919  48.189316  143.83  146.6400  143.7700   
2022-12-29 05:00:00+00:00   789947  48.992500  148.15  148.5100  145.1378   
2022-12-30 05:00:00+00:00   710112  49.092991  146.55  147.7900  144.7400   

                                                                    
                          trade_count    open   volume        vwap  
timestamp                                                           
2022-12-23 05:00:00+00:00       20025  144.51  1017604  145.236784  
2022-12-27 05:00:00+00:00       19294  145.91   957871  145.322438  
2022-12-28 05:00:00+00:00       19669  145.18  1443916  144.409503  
2022-12-29 05:00:00+00:00       21072  145.20  1298851  147.959023  
2022-12-30 05:00:00+00:00       22281  147.20  1249518  146.129091  

[5 rows x 3521 columns]

I want to manually check the data to ensure the first and last values align with each other.

In [23]:
# First and last tickers in the above dataframe
tickers = ['A', 'ZTS']

# Get daily closing prices for these test values. 
testing_df2 = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Display sample data
display(testing_df2.head())
display(testing_df2.tail())


,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-01-03 05:00:00+00:00,156.48,159.44,153.93,24320,159.00,1855491,155.916852,A
2022-01-04 05:00:00+00:00,151.19,155.63,149.70,34709,155.49,2510184,151.522046,A
2022-01-05 05:00:00+00:00,148.60,153.10,148.53,27423,150.83,2966289,149.705613,A
2022-01-06 05:00:00+00:00,149.12,149.96,145.58,34443,148.85,2502203,148.493000,A
2022-01-07 05:00:00+00:00,145.15,149.73,145.09,28610,149.12,2415233,146.271401,A


,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-12-23 05:00:00+00:00,145.76,145.8920,143.5400,20025,144.51,1017604,145.236784,ZTS
2022-12-27 05:00:00+00:00,145.30,146.1474,143.5700,19294,145.91,957871,145.322438,ZTS
2022-12-28 05:00:00+00:00,143.83,146.6400,143.7700,19669,145.18,1443916,144.409503,ZTS
2022-12-29 05:00:00+00:00,148.15,148.5100,145.1378,21072,145.20,1298851,147.959023,ZTS
2022-12-30 05:00:00+00:00,146.55,147.7900,144.7400,22281,147.20,1249518,146.129091,ZTS


Upon manual inspection, the values seem to align.

In [24]:
# Now capture the "close" column for each symbol's dataframe



In [25]:
# Use that "close" column to generate daily percent returns over the entirety of 2022.

In [26]:
# DATA VISUALIZATION 
# Plot those daily returns. Explain overall trend of 2022.
# Plot the volume. Try to add that below the graph.
# Plot the VWAP? Explain what that is.


In [27]:
# DATA ANALYSIS
# Plot Returns versus sector
# Plot returns versus population of location
# Plot 


In [28]:
# GET GEOSPACIAL DATA
# We are interested in US companies that release 'hot' news.
# Where in the US are these companies located?
# We get their location, and we should be able to translate the "name" of that location into "coordinates"
# Once it is in "coordinates", we can plot that point on the Geo map using HVplot


# Frist step would be to get the locations of each company.

# Step 2 is to get access to a dataset that translates cities/towns in the US to coordinates.

# Lastly we can plot those coordinates on the map.

In [29]:
# STEP 2: FIND DATASET TRANSLATING CITY NAME TO COORDINATES
# We can use this API: "GEOCODING API" https://openweathermap.org/api/geocoding-api 
# Here is a description from their website: 

# "Geocoding API is a simple tool that we have developed to ease the search for locations while working with geographic names and coordinates.
# Supporting API calls by geographical coordinates is the most accurate way to specify any location, that is why this method is integrated in all OpenWeather APIs. 
# However, this way is not always suitable for all users.
# Geocoding is the process of transformation of any location name into geographical coordinates, and the other way around (reverse geocoding).
# OpenWeather’s Geocoding API supports both the direct and reverse methods, working at the level of city names, areas and districts, countries and states:
# Direct geocoding converts the specified name of a location or zip/post code into the exact geographical coordinates;
# Reverse geocoding converts the geographical coordinates into the names of the nearby locations."

# We will use direct geocoding.

import requests
# import requests

# Create variable to hold request url
url = "http://api.openweathermap.org/geo/1.0/direct?q=Oakland&limit=5&appid=628878cc6edbe70ff1805be47e1af1ea"

# Add format specifier to request url
#url = url + "?format=json"

# Execute get request
requests.get(url)


# Execute GET request and store response
# response_data = requests.get(url)
# q	required	City name, state code (only for the US) and country code divided by comma. Please use ISO 3166 country codes.
# appid	required	Your unique API key (you can always find it on your account page under the "API key" tab)
# limit	optional	Number of the locations in the API response (up to 5 results can be returned in the API response)

# myCityNameVariable = ""
# for this many rows in our S&P 500 data
#     get the name of city
#     set that name to myCityNameVariable
#     put that myCityNameVariable into the link
#     make that API call

# http://api.openweathermap.org/geo/1.0/direct?q=myCityNameVariable&limit={limit}&appid={API key}


# http://api.openweathermap.org/geo/1.0/direct?q=London&limit=5&appid={API key}

#http://api.openweathermap.org/geo/1.0/direct?q=Oakland&limit=5&appid={3fcc50c01f481adc6307ae95cede2362}

<Response [200]>